In [10]:
from trees import TreeMaker
from wrappers import GPTneoX_DenseWrapper, ActivationWrapper

import torch
import numpy as np

In [2]:
# Install 'desc2025.xml' from MeSH website
tm = TreeMaker('desc2025.xml')

In [3]:
key = 'A06.300'

G, names = tm.tree_from_key(key)
print(names)

('Endocrine Glands', 'Adrenal Glands', 'Gonads', 'Islets of Langerhans', 'Parathyroid Glands', 'Pineal Gland', 'Pituitary Gland', 'Pituitary-Adrenal System', 'Thyroid Gland', 'Adrenal Cortex', 'Adrenal Medulla', 'Ovary', 'Pituitary Gland, Anterior', 'Pituitary Gland, Posterior', 'Testis', 'Somatostatin-Secreting Cells', 'Glucagon-Secreting Cells', 'Insulin-Secreting Cells', 'Pancreatic Polypeptide-Secreting Cells', 'Pituitary Gland, Intermediate', 'Corpus Luteum', 'Ovarian Follicle', 'Leydig Cells', 'Zona Fasciculata', 'Zona Glomerulosa', 'Zona Reticularis', 'Corticotrophs', 'Gonadotrophs', 'Lactotrophs', 'Somatotrophs', 'Thyrotrophs', 'Melanotrophs', 'Sympathoadrenal System', 'Granulosa Cells', 'Luteal Cells', 'Theca Cells', 'Follicular Fluid', 'Cumulus Cells')


In [4]:
model = "EleutherAI/pythia-70m-deduped"
layer_num = 3

wrapper = ActivationWrapper(model)
layer = wrapper.make_layer_wrapper(layer_num, 'mlp')

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [12]:
all_acts, all_toks = layer.batch_activations(names)
last_acts, last_toks = layer.batch_activations(names, tokens='last')

distances = torch.cdist(last_acts, last_acts, p=2)

torch.Size([38, 38])
